## TCP Socket Server

Please google "berkeley socket programming" first. You can find a diagram that describes how to use socket. For a server, it includes the following steps. We will explain them one by one.

* socket
* bind
* listen
* accept
* recv, sned (may repeat multiple times)
* close socket

In [1]:
# import the socket library provided by Python

import socket

In [2]:
# multiple threads

import threading

In [3]:
import requests

In [4]:
import time  

### socket

In [5]:
# create socket instance
# socket.AF_INET is a constant value that indicates I want to use IP (Internet Protocol) as my L3 protocol
# socket.SOCK_STREAM is a constant value that indicates I want to use TCP as my L4 protocol

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

### bind

In [6]:
# ask the OS to bind the created socket to user-specified parameters: (IP, TCP port)
# the 1st parameter is the binded IP, the 2nd parameter is the binded port number
# So that when the OS receives a datagram, the OS knows how to demux the datagram to the corresponding application 

pars = ('127.0.0.1', 80) # you can change the server port to whatever you want
s.bind(pars)

### listen

In [7]:
# become a server socket
# it makes this python program waiting for receiving message
# listen() function has one parameter that limits how many clients can be connected to this server
# we set it to 5 without any reason
# you can change it to any number you want, as long as you have sufficient resources (computing, memory)

s.listen(5)

### recv, send and close function
Note: it is a function declaration. This function will be called after 'accept'

In [8]:
# a new thread is created for every new accepted client
# every new thread starts from the function below

def serveClient(clientsocket, address):
    
    # we need a loop to continuously receive messages from the client
    while True:
        # then receive at most 1024 bytes message and store these bytes in a variable named 'data'
        data = clientsocket.recv(1024)
        print("from client", data)
        
        # data: GET /get.html HTTP/1.1\r\n
        data_decode = data.decode()
        start = data_decode.find('/',0)
        end = data_decode.find('l',start)
        end = end+1
        print(data_decode[start:end])
        
        # time
        gmt_now = time.strftime("%a, %d %b %Y %I:%M:%S GMT", time.gmtime())
                
        # GET 200 OK
        if data_decode[start:end] == "/get.html":
            response_data = b'<html><body>good.html<body><html>'
            length = len(response_data)            
            response_status = b'HTTP/1.0 200 OK\r\n'
            response_header= '%s%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                       gmt_now,
                                                       '\r\n',
                                                       'Server: localhost\r\n',
                                                       'Connection: close\r\n',
                                                       'Accept-Ranges: bytes\r\n',
                                                       'Content-Type: text/html; charset=UTF-8\r\n',
                                                       'Content-Length: ',
                                                       length,
                                                       '\r\n')
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)

            print("[200 OK Response]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)
            break
    
        # GET 301 Moved Permanently      
        elif data_decode[start:end] == "/redirect.html":
            response_data = b'<html><body>Moved to get.html<body><html>'
            length = len(response_data)  
            response_status = b'HTTP/1.0 301 Moved Permanently\r\n'
            clientsocket.send(response_status) 
            response_header= '%s%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                       gmt_now,
                                                       '\r\n',
                                                       'Server: localhost\r\n',
                                                       'Connection: close\r\n',
                                                       'Location: http://127.0.0.1/get.html\r\n',
                                                       'Content-Type: text/html\r\n',
                                                       'Content-Length: ',
                                                       length,
                                                       '\r\n',
                                                      )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)  
            
            print("[301 Redirect Response]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)
            break
            
        # GET 404 Not Found      
        elif data_decode[start:end] == "/notfound.html":
            response_data = b'<html><body>404 Not Found<body><html>'
            length = len(response_data)            
            response_status = b'HTTP/1.0 404 Not Found\r\n'
            response_header= '%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                     gmt_now,
                                                     '\r\n',
                                                     'Server: localhost\r\n',
                                                     'Connection: close\r\n',
                                                     'Content-Type: text/html; charset=UTF-8\r\n',
                                                     'Content-Length: ',
                                                     length,
                                                     '\r\n',
                                                    )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)

            print("[404 Not Found Response]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)
            break
            
        # HEAD    
        elif data_decode[start:end] == "/head.html": 
            response_data = b'<html><body>head.html<body><html>'
            length = len(response_data) 
            response_status = b'HTTP/1.0 200 OK\r\n'          
            response_header= '%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                     gmt_now,
                                                     '\r\n',
                                                     'Server: localhost\r\n',
                                                     'Connection: close\r\n',
                                                     'Content-Type: text/html; charset=UTF-8\r\n',
                                                     'Content-Length: ',
                                                     length,
                                                     '\r\n',
                                                    )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            
            print("[HEAD]: ")
            print(response_status)
            print(response_header_b)
#             print(response_data)
            break
            
        # POST    
        elif data_decode[start:end] == "/post.html":  
            id_msg = data_decode.find('id=',0)
            id_beg = data_decode.find('=',id_msg)+1
            id_end = id_beg+9
            print(data_decode[id_beg:id_end])
            
            # test
#             msg_or = 'POST /post.html HTTP/1.1\r\nHost: 127.0.0.1\r\nid=105204027\r\n'
#             msg_or = 'POST /post.html HTTP/1.1\r\nHost: 127.0.0.1\r\n\r\n'
#             msg = msg_or.find('id=',0)
#             m_beg = msg_or.find('=',msg)+1
#             m_end = m_beg+9
#             print(msg_or[m_beg:m_end])
#             if msg == -1:
#                 stu_id = 'You did not post your student id correctly.'
#             else:
#                 stu_id = msg_or[m_beg:m_end]
            
            if id_msg == -1:
                stu_id = 'You did not post your student id correctly.'
            else:
                stu_id = data_decode[id_beg:id_end]
            data = '<html><body>%s<body><html>' % (stu_id)
            response_data = data.encode(encoding="utf-8")            
            length = len(response_data)            
            response_status = b'HTTP/1.0 200 OK\r\n'
            response_header= '%s%s%s%s%s%s%s%d%s' % ('Date: \r\n',
                                                     gmt_now,
                                                     '\r\n',
                                                     'Server: localhost\r\n',
                                                     'Connection: close\r\n',
                                                     'Content-Type: text/html; charset=UTF-8\r\n',
                                                     'Content-Length: ',
                                                     length,
                                                     '\r\n',
                                                    )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)

            print("[POST]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)            
            break

        else:
            print("false")
            break
            
        clientsocket.close()

   
        # if the client sends some termination message to the server, the server close the socket 
#         if data == b'close':
#             clientsocket.close()
#             break

### accept

In [9]:
# since at most we can serve many clients (5 in this example), we need a way to distinguish them 
# as mentioned in the class, TCP use 4-tuple (src IP, dst IP, src port, dst port) to distinguish a socket
# we use accept() function to confirm that we connect to the client socket
# and accept() function will return the client's socket instance and IP
# we need a loop to keep accepting new clients (until 5 clients are accepted)

while True:
    # accept a new client and get it's information
    (clientsocket, address) = s.accept()
    
    # create a new thread to serve this new client
    # after the thread is created, it will start to execute 'target' function with arguments 'args' 
    threading.Thread(target = serveClient, args = (clientsocket, address)).start()
    
    print("Accepted")

Accepted
Accepted
from client b'GET /head.html HTTP/1.1\r\nHost: 127.0.0.1\r\nConnection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\r\nPurpose: prefetch\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mode: navigate\r\nSec-Fetch-User: ?1\r\nSec-Fetch-Dest: document\r\nAccept-Encoding: gzip, deflate, br\r\nAccept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7\r\n\r\n'
/head.html
[HEAD]: 
b'HTTP/1.0 200 OK\r\n'
b'Date: Sat, 05 Dec 2020 12:45:16 UTC\r\nServer: localhost\r\nConnection: close\r\nContent-Type: text/html; charset=UTF-8\r\nContent-Length: 33\r\n'
from client b'GET /head.html HTTP/1.1\r\nHost: 127.0.0.1\r\nConnection: keep-alive\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS

KeyboardInterrupt: 